# Cleaning Global Knowledge Portal Data

In [102]:
# For multiple output per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [79]:
#DATASET_FOLDER = '/media/data-nvme/dev/datasets/WorldBank/'
DATASET_FOLDER = '../../datasets/precipitation/'

SPARK_MASTER = 'spark://192.168.0.9:7077'
APP_NAME = 'Merge PRCP GKP'
input_folder = DATASET_FOLDER
output = DATASET_FOLDER + '../wb_gkp_precipitation'

In [80]:
import os
import pandas as pd

In [81]:
rcp_projection = ['rcp26', 'rcp45','rcp60', 'rcp85']

def abreviation2nombre(abr):
        lst_abr = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
        return(lst_abr.index(abr)+1)

def read_onefile(filename):
    country_error_search_string = ['The', 'State of', 'United Republic of', 'Democratic People’s Republic of', 'Republic of']
    df = pd.read_csv(filename, sep=r', ', engine='python')
    if 'historical' in filename:
        # S'il y a un problème de vigule on nettoit le dataframe
        if 'Country' in df.columns and df.Country.all() in country_error_search_string:
            df.reset_index(inplace=True)
            # On récuppère le nom réel
            df['new_Country'] = df[['Statistics', 'Country']].apply(lambda x: x[0]+', '+x[1], axis=1)
            df.drop('Country', axis=1, inplace=True)
            df.rename(columns={'index': 'Rainfall - (MM)',
                               'Rainfall - (MM)': 'Year',
                               'Year': 'Statistics', 
                               'Statistics': 'tmp',
                               'new_Country': 'Country'}, inplace=True)
            df.drop('tmp', axis=1, inplace=True)
    else:
        for rcp in rcp_projection:
            if rcp in filename:
                df['RCP'] = rcp
        #print(entry.name, ':', df.columns)
        # S'il y a un problème de vigule on nettoit le dataframe
        if 'Country' in df.columns and df.Country.all() in country_error_search_string:
            df.reset_index(inplace=True)
            # On récuppère le nom réel
            df['new_Country'] = df[['Statistics', 'Country']].apply(lambda x: x[0]+', '+x[1], axis=1)
            df.drop('Country', axis=1, inplace=True)

            df.rename(columns={'index': 'Monthly Precipitation - (MM)',
                               'Monthly Precipitation - (MM)': 'Year',
                               'Year': 'Model', 
                               'Model': 'Statistics',
                               'Statistics': 'tmp',
                               'new_Country': 'Country'}, inplace=True)
            df.drop('tmp', axis=1, inplace=True)    
    return df

def gen_dataset():
    '''Créer les deux fichier : historique_precipitation_clean.csv et projection_precipitation_clean.csv
    '''
    
    dir_precipitation = DATASET_FOLDER
    df_hist = pd.DataFrame()
    df_pred = pd.DataFrame()
    for i, entry in enumerate(os.scandir(dir_precipitation)):
        print(f'\r{i+1}', end='')
        filename = entry.path
        if '.csv' in entry.name:
            try:
                df = read_onefile(filename)
                if 'historical' in entry.name:
                    df_hist = df_hist.append(df)
                else:
                    df_pred = df_pred.append(df)
            except:
                print('ERROR reading', filename)
    df_hist.to_csv(f'{DATASET_FOLDER}../historical_precipitation_clean2.csv', index=False)
    df_pred.to_csv(f'{DATASET_FOLDER}../projection_precipitation_clean2.csv', index=False)

In [82]:
#read_onefile('../../datasets/precipitation/projection_2060_2079_GAB_rcp85_mavg.csv') # Gabon
#read_onefile('../../datasets/precipitation/projection_2080_2099_NGA_rcp26_mavg.csv') # Nigeria
#read_onefile('../../datasets/precipitation/projection_2040_2059_UGA_rcp60_mavg.csv') # Uganda
#df = read_onefile('../../datasets/precipitation/projection_2060_2079_GMB_rcp26_mavg.csv') # Gambia
df.Country.all()

'The'

In [83]:
gen_dataset()

3476

# Historical cleaning

In [84]:
df_hist = pd.read_csv(f'{DATASET_FOLDER}../historical_precipitation_clean2.csv')
df_hist

,Rainfall - (MM),Year,Statistics,Country,ISO3
0,92.41310,1901,Jan Average,Malta,MLT
1,88.87840,1901,Feb Average,Malta,MLT
2,14.28420,1901,Mar Average,Malta,MLT
3,13.64250,1901,Apr Average,Malta,MLT
4,26.48340,1901,May Average,Malta,MLT
...,...,...,...,...,...
272827,69.62290,2016,Aug Average,Eritrea,ERI
272828,40.15910,2016,Sep Average,Eritrea,ERI
272829,16.66870,2016,Oct Average,Eritrea,ERI
272830,4.43639,2016,Nov Average,Eritrea,ERI


In [91]:
#     # On créé une variable pour pouvoir faire le tri sur les mois
df_hist['Month'] = df_hist['Statistics'].str[:4].apply(lambda x: abreviation2nombre(x.strip()))
df_hist.sort_values(['Country','Year','Month'], inplace=True)    
df_hist.Month.value_counts()

12    22736
11    22736
10    22736
9     22736
8     22736
7     22736
6     22736
5     22736
4     22736
3     22736
2     22736
1     22736
Name: Month, dtype: int64

In [86]:
df_hist[df_hist.Statistics=='Korea']

,Rainfall - (MM),Year,Statistics,Country,ISO3


In [87]:
#!ls $DATASET_FOLDER*KOR*

In [88]:
#!head ../../datasets/precipitation/historical_1901-2016_KOR__mavg.csv

In [93]:
df_hist.to_csv(f'{DATASET_FOLDER}../historical_precipitation_clean_2020-12-01.csv.gz', index=False ,compression='gzip')

# Projections cleanng

In [94]:
df_pred = pd.read_csv(f'{DATASET_FOLDER}../projection_precipitation_clean2.csv')
df_pred

,Monthly Precipitation - (MM),Year,Model,Statistics,Country,ISO3,RCP,Expected Daily Rainfall Maximum in 25 Years (25-yr Return Level) - (MM),Largest Single Day Rainfall - (MM),Expected Daily Rainfall Maximum in 10 Years (10-yr Return Level) - (MM),Expected 5-day Cumulative Rainfall Maximum in 10 Years (10-yr Return Level) - (MM),Rainfall Amount from Very Wet Days - (Percentage),Largest 5-day Cumulative Rainfall - (MM),Expected Largest Monthly Rainfall Amount in 25 Years (25-yr Return Level) - (MM),Number of Days with Rainfall > 50mm - (Days),Number of Days with Rainfall > 20mm - (Days),Expected Largest Monthly Rainfall Amount in 10 Years (10-yr Return Level) - (MM)
0,61.232018,2040-2059,bcc_csm1_1,Jan Average,Italy,ITA,rcp60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,44.391956,2040-2059,bcc_csm1_1,Feb Average,Italy,ITA,rcp60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,40.673943,2040-2059,bcc_csm1_1,Mar Average,Italy,ITA,rcp60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,41.089370,2040-2059,bcc_csm1_1,Apr Average,Italy,ITA,rcp60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,44.656418,2040-2059,bcc_csm1_1,May Average,Italy,ITA,rcp60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
750805,2.608093,2060-2079,Ensemble (90th Percentile),Aug Average,Israel,ISR,rcp60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
750806,5.112417,2060-2079,Ensemble (90th Percentile),Sep Average,Israel,ISR,rcp60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
750807,5.534160,2060-2079,Ensemble (90th Percentile),Oct Average,Israel,ISR,rcp60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
750808,2.549516,2060-2079,Ensemble (90th Percentile),Nov Average,Israel,ISR,rcp60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [95]:
df_pred.Statistics.value_counts()

Aug Average       60192
Oct Average       60192
May Average       60192
Sep Average       60192
Jun Average       60192
Mar Average       60192
Jan Average       60192
Feb Average       60192
Dec Average       60192
Apr Average       60192
Jul Average       60192
Nov Average       60192
Annual Anomaly     4518
Nov Anomaly        2065
Sep Anomaly        2043
Dec Anomaly        2035
Apr Anomaly        2032
Mar Anomaly        2031
Oct Anomaly        2029
Jan Anomaly        1980
Jun Anomaly        1977
Feb Anomaly        1973
May Anomaly        1965
Aug Anomaly        1932
Jul Anomaly        1926
Name: Statistics, dtype: int64

In [100]:
import numpy as np
#     # On créé une variable pour pouvoir faire le tri sur les mois
df_pred['Month'] = df_pred['Statistics'].str[:4].apply(lambda x: abreviation2nombre(x.strip()) if x != 'Annu' else np.NaN)
df_pred.sort_values(['Country','Year','Model','Month'], inplace=True)
df_pred.Month.value_counts()

11.0    62257
9.0     62235
12.0    62227
4.0     62224
3.0     62223
10.0    62221
1.0     62172
6.0     62169
2.0     62165
5.0     62157
8.0     62124
7.0     62118
Name: Month, dtype: int64

In [96]:
df_pred[df_pred.Statistics=='Gambia']

,Monthly Precipitation - (MM),Year,Model,Statistics,Country,ISO3,RCP,Expected Daily Rainfall Maximum in 25 Years (25-yr Return Level) - (MM),Largest Single Day Rainfall - (MM),Expected Daily Rainfall Maximum in 10 Years (10-yr Return Level) - (MM),Expected 5-day Cumulative Rainfall Maximum in 10 Years (10-yr Return Level) - (MM),Rainfall Amount from Very Wet Days - (Percentage),Largest 5-day Cumulative Rainfall - (MM),Expected Largest Monthly Rainfall Amount in 25 Years (25-yr Return Level) - (MM),Number of Days with Rainfall > 50mm - (Days),Number of Days with Rainfall > 20mm - (Days),Expected Largest Monthly Rainfall Amount in 10 Years (10-yr Return Level) - (MM)


In [112]:
df_pred.columns
len(df_pred.columns)

Index(['Monthly Precipitation - (MM)', 'Year', 'Model', 'Statistics',
       'Country', 'ISO3', 'RCP',
       'Expected Daily Rainfall Maximum in 25 Years (25-yr Return Level) - (MM)',
       'Largest Single Day Rainfall - (MM)',
       'Expected Daily Rainfall Maximum in 10 Years (10-yr Return Level) - (MM)',
       'Expected 5-day Cumulative Rainfall Maximum in 10 Years (10-yr Return Level) - (MM)',
       'Rainfall Amount from Very Wet Days - (Percentage)',
       'Largest 5-day Cumulative Rainfall - (MM)',
       'Expected Largest Monthly Rainfall Amount in 25 Years (25-yr Return Level) - (MM)',
       'Number of Days with Rainfall > 50mm - (Days)',
       'Number of Days with Rainfall > 20mm - (Days)',
       'Expected Largest Monthly Rainfall Amount in 10 Years (10-yr Return Level) - (MM)',
       'Month'],
      dtype='object')

18

In [114]:
rename = ['monthly_prcp_mm', 'year', 'model', 'statistics',
       'country', 'ISO3', 'projection_rcp',
       'daily_rain_max_25_years_mm',
       'largest_single_day_rain_mm',
       'daily_rain_max_10_years_mm',
       '5-day_rain_sum_max_10_years_mm',
       'rain_from_very_wet_days_percent',
       'largest_5-day_rain_sum_mm',
       'largest_month_rain_25_years_mm',
       'nb_days_with_rain_>_50mm',
       'nb_days_with_rain_>_20mm',
       'largest_month_rain_10_years_mm',
       'month']
len (rename)
df_pred.columns = rename
df_pred.columns

18

Index(['monthly_prcp_mm', 'year', 'model', 'statistics', 'country', 'ISO3',
       'projection_rcp', 'daily_rain_max_25_years_mm',
       'largest_single_day_rain_mm', 'daily_rain_max_10_years_mm',
       '5-day_rain_sum_max_10_years_mm', 'rain_from_very_wet_days_percent',
       'largest_5-day_rain_sum_mm', 'largest_month_rain_25_years_mm',
       'nb_days_with_rain_>_50mm', 'nb_days_with_rain_>_20mm',
       'largest_month_rain_10_years_mm', 'month'],
      dtype='object')

In [115]:
df_pred.to_csv(f'{DATASET_FOLDER}../projection_precipitation_clean_2020-12-01.csv', index=False)